<a href="https://colab.research.google.com/github/larissavvsous/Aprendizado-de-Maquina-e-Reconhecimento-de-Padroes/blob/main/Larissa_sudoku_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm

In this project, the aim is to solve a sudoku puzzle using the genetic algorithm

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import random as rndm
import time

## Part 1: Defining Genes and Chromosomes

A gene is the value of a row and is a permutation of set {1 ... 9}. A chromosome consists of 9 genes, each gene representing a row of the actual sudoku puzzle.

In [18]:
def make_gene(initial=None):
    if initial is None:
        initial = [0] * 16
    mapp = {}
    gene = list(range(1, 17))
    rndm.shuffle(gene)
    for i in range(16):
        mapp[gene[i]] = i
    for i in range(16):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]
    return gene

In [19]:
def make_chromosome(initial=None):
    if initial is None:
        initial = [[0] * 16] * 16
    chromosome = []
    for i in range(16):
        chromosome.append(make_gene(initial[i]))
    return chromosome

## Part 2: Making First Generation

In this part, a function is implemented to randomly create a population.

In [20]:
def make_population(count, initial=None):
    if initial is None:
        initial = [[0] * 16] * 16
    population = []
    for _ in range(count):
        population.append(make_chromosome(initial))
    return population

## Part 3: Fitness Function

The fitness function calculates how "fit" a chromosome (puzzle) is based on:
- For each column: Subtract (number of times a number is seen) - 1 from the fitness for that number
- For each 3x3 square: Subtract (number of times a number is seen) - 1 from the fitness for that number
The higher the fitness, the closer the puzzle is to being solved.

In [21]:
def get_fitness(chromosome):
    """Calculate the fitness of a chromosome (puzzle)."""
    fitness = 0
    for i in range(16): # For each column
        seen = {}
        for j in range(16): # Check each cell in the column
            if chromosome[j][i] in seen:
                seen[chromosome[j][i]] += 1
            else:
                seen[chromosome[j][i]] = 1
        for key in seen: # Subtract fitness for repeated numbers
            fitness -= (seen[key] - 1)
    for m in range(4): # For each 3x3 square
        for n in range(4):
            seen = {}
            for i in range(4 * n, 4 * (n + 1)):  # Check cells in 3x3 square
                for j in range(4 * m, 4 * (m + 1)):
                    if chromosome[j][i] in seen:
                        seen[chromosome[j][i]] += 1
                    else:
                        seen[chromosome[j][i]] = 1
            for key in seen: # Subtract fitness for repeated numbers
                fitness -= (seen[key] - 1)
    return fitness

In [22]:
ch = make_chromosome()
print(get_fitness(ch))


def pch(ch):
    for i in range(16):
        for j in range(16):
            print(ch[i][j], end=" ")
        print("")


-176


## Part 4: Crossover and Mutation

In this part, the crossover and mutation function is implemented to help in determining the next generation.

### Crossover

the crossover function takes two chromosomes as input and makes two new chromosomes by combining them. This crossover function decides the parent of each gene separately, so the result is independent of the location of the genes.

In [23]:
def crossover(ch1, ch2):
    new_child_1 = []
    new_child_2 = []
    for i in range(16):
        x = rndm.randint(0, 1)
        if x == 1:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
        elif x == 0:
            new_child_2.append(ch1[i])
            new_child_1.append(ch2[i])
    return new_child_1, new_child_2

### Mutation

In [24]:
def mutation(ch, pm, initial):
    for i in range(16):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            ch[i] = make_gene(initial[i])
    return ch

## Part 5: Implementing The Genetic Algorithm

In this part, we use the components defined in previous steps to write the genetic algorithm.

Read the puzzle from the given address


In [25]:
def read_puzzle(address):
    puzzle = []
    f = open(address, 'r')
    for row in f:
        temp = row.split()
        puzzle.append([int(c) for c in temp])
    return puzzle

Get the mating pool using the random method

In [26]:
def r_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    fitness_list.sort()
    weight = list(range(1, len(fitness_list) + 1))
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weight)[0]
        pool.append(ch[1])
    return pool


Get the mating pool using the weighted method

In [27]:
def w_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    weight = [fit[0] - fitness_list[0][0] for fit in fitness_list]
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weights=weight)[0]
        pool.append(ch[1])
    return pool


Generate the offsprings from the mating pool

In [28]:
def get_offsprings(population, initial, pm, pc):
    new_pool = []
    i = 0
    while i < len(population):
        ch1 = population[i]
        ch2 = population[(i + 1) % len(population)]
        x = rndm.randint(0, 100)
        if x < pc * 100:
            ch1, ch2 = crossover(ch1, ch2)
        new_pool.append(mutation(ch1, pm, initial))
        new_pool.append(mutation(ch2, pm, initial))
        i += 2
    return new_pool

In [29]:
# Population size
POPULATION = 1000

# Number of generations
REPETITION = 1000

# Probability of mutation
PM = 0.1

# Probability of crossover
PC = 0.95

# Main genetic algorithm function
def genetic_algorithm(initial_file):
    initial = read_puzzle(initial_file)
    population = make_population(POPULATION, initial)
    for _ in range(REPETITION):
        mating_pool = r_get_mating_pool(population)
        rndm.shuffle(mating_pool)
        population = get_offsprings(mating_pool, initial, PM, PC)
        fit = [get_fitness(c) for c in population]
        m = max(fit)
        if m == 0:
            return population
    return population

Run the algorithm and time it

In [15]:
tic = time.time()
r = genetic_algorithm("/content/drive/MyDrive/anexos/4x4-01.txt")
toc = time.time()
print("time_taken: ", toc - tic)
fit = [get_fitness(c) for c in r]
m = max(fit)
print(max(fit))

# Print the chromosome with the highest fitness
for c in r:
    if get_fitness(c) == m:
        pch(c)
        break

time_taken:  418.91910195350647
-61
14 9 4 3 10 5 6 13 15 11 8 12 1 2 7 16 
7 4 5 12 14 9 1 15 16 13 10 6 11 3 2 8 
5 8 6 15 3 11 16 12 7 10 9 1 2 4 14 13 
16 10 2 7 15 8 11 4 3 5 13 14 9 6 12 1 
6 7 13 2 8 14 11 5 12 1 3 9 4 15 10 16 
10 5 12 4 16 9 13 3 6 15 14 11 1 8 2 7 
14 15 3 16 2 8 7 1 10 4 5 13 12 9 6 11 
1 11 9 3 15 4 12 10 16 8 2 7 6 5 13 14 
12 14 11 5 8 7 3 6 4 9 16 15 13 10 1 2 
7 16 1 2 4 12 10 5 8 11 13 3 14 6 9 15 
13 6 8 10 9 15 2 11 14 7 12 5 4 1 16 3 
4 3 15 11 6 16 9 14 13 2 1 10 8 7 12 5 
11 5 10 9 7 4 8 15 1 16 6 2 3 13 14 12 
4 2 7 1 9 13 15 16 10 12 11 3 5 14 8 6 
3 12 16 6 1 10 14 11 5 13 7 8 15 9 4 2 
8 13 15 1 5 3 4 2 9 14 12 6 7 10 11 16 


## Problems with Genetic Algorithm

The algorithm may encounter a challenge when it becomes trapped at a local maximum and the desired solution, such as in a Sudoku problem, cannot be achieved. One approach to address this issue is to initially increase the mutation rate, although this can have its own drawbacks as mentioned earlier. Alternatively, a function can be defined to identify local maximums, such as by detecting repeated values, and in such cases, the mutation rate can be increased. Another solution involves increasing the population size or the number of iterations the algorithm runs. Modifying the fitness function or changing the method for selecting individuals in the mating pool are additional strategies. Overall, since the algorithm's nature involves randomness, it is possible to encounter this problem regardless of the chosen method, but running the algorithm multiple times can sometimes be helpful.

## Diferenças entre esse código e o outro

### Para algoritmos genéticos, dependendo da implementação, o tempo de execução pode variar por vários motivos:

- Tamanho da População: O tamanho da população tem um impacto significativo no desempenho dos algoritmos genéticos. Populações maiores podem levar mais tempo para serem processadas porque o número de cromossomos a serem analisados ​​aumenta a cada geração. Muito ou pouco afeta a diversidade genética da população, por isso levará algum tempo para encontrar uma solução.
- Configurações de parâmetros: As configurações dos parâmetros do algoritmo genético, como número máximo de gerações, tamanho da população, variância e taxa de divisão, afetam o tempo de execução e a qualidade da solução obtida.

Portanto, o primeiro código (código que utiliza diretamente o algoritmo genético para resolver o Sudoku) é mais rápido que o segundo código (código que utiliza uma biblioteca externa através do módulo object.sudoku_genetics).